In [5]:
# import needed packages
import json
import pandas as pd
import requests
import sqlite3
import requests
import urllib
import sqlalchemy
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

In [7]:
# read the csv file
fruit_url = "https://www.ers.usda.gov/webdocs/DataFiles/51035/Fruit-Prices-2022.csv"
fruit = pd.read_csv(fruit_url)

# read API data
usda_key = "fyMj8OX6LXYj6hFYFxRtPX0xY6VRzX8WepkX2Xjh"
url_foods = "https://api.nal.usda.gov/fdc/v1/foods/search"
headers = {
    'x-api-key': usda_key
    }

#usda api https://api.nal.usda.gov/fdc/v1/foods/search

In [9]:
# prompt for taking api or csv data source
print("Would you like to examine average fruit prices in the US (1) \nor explore food data from the Department of Agriculture (2)? \nPlease input a 1 or 2 below:")
data_input = input()
# prompt for what kind of final ouput
print("Would you like data as a CSV (1), JSON (2), or SQL database (3)?" + "\n" +
     "Please input a 1,2, or 3 below")
type_input = input()
if (data_input == "1"): # CSV source
    if (type_input == "1"): # CSV, no transformation besides pandas  
        print("You have requested to see average USA fruit prices as a CSV file.\n" + 
             "There are " + str(fruit_csv.shape[0]) + " fruits in this dataset. And " + str(fruit_csv.shape[1]) + " columns.")
        display(fruit.head())
        #option to delete unecessary column
        print("Would you like to remove the Yield column? Type 1 for yes, press enter for no")
        yield_col = input()

        if (yield_col== "1"):
            del fruit['Yield']

        print("Processing is complete. Name of file: fruit.csv\nThere are " + str(fruit.shape[0]) + " fruits in this dataset and " + str(fruit.shape[1])+" columns.")
        fruit.to_csv('fruit.csv')
        fruit_csv = pd.read_csv('fruit.csv')
        display(fruit_csv.head())
        
    if (type_input == "2"): # CSV to JSON
        
        print("You have requested to see average USA fruit prices in a JSON format.\n "
              + "\n There are " + str(fruit.shape[0]) + " fruits in this dataset and " + str(fruit.shape[1])+" columns.")
        display(fruit.head())
        #option to delete unecessary column
        print("Would you like to remove the Yield column? Type 1 for yes, press enter for no")
        yield_col = input()

        if (yield_col== "1"):
            del fruit['Yield']
        
        print("Processing is complete. Name of file: fruit_prices_json.txt\n"
             + "There are " + str(fruit.shape[0]) + " fruits in this dataset and " + str(fruit.shape[1])+" columns.")
        json_fruit = fruit.to_json(orient='records', indent=4)

        jfruit = json.loads(json_fruit)

        with open('fruit_prices_json.txt', 'w') as f:
            f.write(json_fruit)
        
        print(jfruit[0:6])
        
    
    if (type_input == "3"): # CSV to SQL
        print("You have requested to see average USA fruit prices as a SQL database.")
        display(fruit.head())
        #option to delete unecessary column
        print("Would you like to remove the Yield column? Type 1 for yes, press enter for no")
        yield_col = input()
        
        conn = sqlite3.connect('fruits.db')
        cur = conn.cursor()
        
                        # Create a table
        cur.execute(''' 
                            CREATE TABLE IF NOT EXISTS fruit_price (
                                Fruit TEXT,
                                Form TEXT,
                                RetailPrice FLOAT,
                                RetailPriceUnit TEXT,
                                Yield FLOAT,
                                CupEquivalentSize FLOAT,
                                CupEquivalentUnit TEXT,
                                CupEquivalentPrice FLOAT
                                
                            )
                        ''')
        conn.commit() 
        #fruit.to_sql('food_price', conn, if_exists='replace')
        for row in fruit.itertuples(index=False, name=None):
            cur.execute('INSERT INTO fruit_price VALUES (?, ?,?,?,?,?,?,?)', row)
        if (yield_col == "1"):
            del fruit['Yield']
            cur.execute('''
            ALTER TABLE fruit_price DROP Yield;
                '''
            )
       #check that sql data is fetched properly
        try:
            joined_data = cur.fetchall()
               
            for row in range(5):
                print(joined_data[row])
            print("Processing is complete. Database Name: fruits.db \n There are " + str(fruit.shape[0]) + " fruits in this dataset and " + str(fruit.shape[1])+" columns.")
        conn.close()
        except:
            print ("Error: unable to fetch data")
            conn.close()
        

                    
        
        
        

elif (data_input == "2"): # API source
    print("Please type the food you want to learn about")
    food_input = input()
    querystring = {"query":food_input}
    #check for functioning API request
    if food_response.status_code == 200:
        print("API request successful!")
    else:
        print(f"Failed to retrieve data: {response.status_code}")

    food_response = requests.request("GET", url_foods, headers=headers, params=querystring)
    food_json = food_response.json() 
    food_ouput = food_json['foods']
    
    food_name = str(food_ouput[0]['description'])
    food_ingred = str(food_ouput[0]['ingredients'])
    food_country_market = str(food_ouput[0]['marketCountry'])
    food_SS = str(food_ouput[0]['servingSize'])
    food_SSU = str(food_ouput[0]['servingSizeUnit'])
    
    if (type_input == "1"): # JSON to CSV
        print("You have requested to see USDA information on " + food_input +  " as a CSV file.")

        #option to delete unecessary column

        print("Name: " + food_name + "\n" + "Ingredients: " + food_ingred +  "\n" + "Country Market: " + food_country_market + 
            "\n" + "Serving Size: " + food_SS + food_SSU + 
              "\nWould you like to delete the ingredients list? Answer 1 for yes, or type in any response for no")
        ingred_input = input()
        if (ingred_input == "1"):
            data = [{'Name': food_name,'Country Market': food_country_market,
                'Serving Size': food_SS, 'Serving Size Unit': food_SSU}]
            
            print("Your dataset has one row and 4 columns" + "\nName: " + food_name + "\n" + "Country Market: " + food_country_market + 
     "\n" + "Serving Size: " + food_SS + food_SSU)
        else:
            data = [{'Name': food_name, 'Ingredients List': food_ingred, 'Country Market': food_country_market,
                'Serving Size': food_SS, 'Serving Size Unit': food_SSU}] 
            print("Your dataset has one row and 5 columns." +"\nName: " + food_name + "\nIngredients: " + food_ingred + "\n" + "Country Market: " + food_country_market + 
     "\n" + "Serving Size: " + food_SS + food_SSU)
        
        
        # Creates DataFrame.
        food_df = pd.DataFrame(data)
        # save file to csv
        food_df.to_csv(food_name +'.csv')
        food_csv = pd.read_csv(food_name +'.csv')
        print("Processing is complete. There are " + str(food_df.shape[0]) + " row(s) in this dataset and " + str(food_df.shape[1])+" column(s)."+
             "\nFile name: " + str(food_name)+".csv")
        display(food_csv.head())
    
    if (type_input == "2"): # JSON, no transformation needed
        print("You have requested to see USDA information on " + food_input +  " in JSON format.")
        #option to delete unecessary column
        print("Name: " + food_name + "\n" + "Ingredients: " + food_ingred +  "\n" + "Country Market: " + food_country_market + 
     "\n" + "Serving Size: " + food_SS + food_SSU + "\nWould you like to delete the ingredients list? Answer 1 for yes, or type in any response for no")

        ingred_input = input()
        if (ingred_input == "1"):
            data = [{'Name': food_name,'Country Market': food_country_market,
                'Serving Size': food_SS, 'Serving Size Unit': food_SSU}]
            print("Your dataset has one row and 4 columns" + "\nName: " + food_name + "\n" + "Country Market: " + food_country_market + 
     "\n" + "Serving Size: " + food_SS + food_SSU)
        else:
            data = [{'Name': food_name, 'Ingredients List': food_ingred, 'Country Market': food_country_market,
                'Serving Size': food_SS, 'Serving Size Unit': food_SSU}] 
            print("Your dataset has one row and 5 columns." +"\nName: " + food_name + "\nIngredients: " + food_ingred + "\n" + "Country Market: " + food_country_market + 
     "\n" + "Serving Size: " + food_SS + food_SSU)
        food_df = pd.DataFrame(data)
        json_food = food_df.to_json(orient='records', indent=4)

        jfood = json.loads(json_food)

        filename = 'USDA_' + str(food_name) + '_json.txt'
        with open(filename, 'w') as f:
            f.write(str(jfood))

        print("Processing is complete. Json txt file name:" + filename)
   
    if (type_input == "3"): # JSON to SQL
        print("You have requested to see USDA information on " + food_input +  " as a SQL database.")
        data = [{'Name': food_name, 'Ingredients List': food_ingred, 'Country Market': food_country_market,
                'Serving Size': food_SS, 'Serving Size Unit': food_SSU}]
        #option to delete unecessary column
        print("Name: " + food_name + "\n" + "Ingredients: " + food_ingred +  "\n" + "Country Market: " + food_country_market + 
     "\n" + "Serving Size: " + food_SS + food_SSU + "\nWould you like to delete the ingredients list? Answer 1 for yes, or type in any response for no")
        ingred_input = input()
        if (ingred_input == "1"):
            data = [{'Name': food_name,'Country Market': food_country_market,
                'Serving Size': food_SS, 'Serving Size Unit': food_SSU}]
            print("Your dataset has one row and 4 columns" + "\nName: " + food_name + "\n" + "Country Market: " + food_country_market + 
                 "\n" + "Serving Size: " + food_SS + food_SSU)
        else:
            data = [{'Name': food_name, 'Ingredients List': food_ingred, 'Country Market': food_country_market,
                'Serving Size': food_SS, 'Serving Size Unit': food_SSU}] 
            print("Your dataset has one row and 5 columns." +"\nName: " + food_name + "\nIngredients: " + food_ingred + "\n" + "Country Market: " + food_country_market + 
                 "\n" + "Serving Size: " + food_SS + food_SSU)
        
        # Creates DataFrame.
        food_df = pd.DataFrame(data)
        
        # Connect to SQLite
        conn = sqlite3.connect('food_sql.db')
        c = conn.cursor()
        
       
        # Insert JSON data into the SQLite table
        food_df.to_sql('food_table', conn, if_exists='replace')
                # Commit and close the connection
                
        c.execute('''SELECT * FROM food_table''')
        #check data was retrieved into sql database
        try:            
            join_data = c.fetchall()
            print(join_data)
            conn.commit()
            conn.close()
            print("Processing is complete. Your data is sotred in the food_sql.db under food_table.")
        except:
            print("Error. Unable to fetch data.")
            conn.commit()
            conn.close()
        
        # Commit and close the connection
        
#not able to process input    
else:
    print("There was an error processing your request. Please try again.")

# output statement that summarizes the user requested
# include stuff like the sortOrder and brandOwner if you have time ok
#ask whether or not they want the raw JSON string ok
# how to display the SQL ????

Would you like to examine average fruit prices in the US (1) 
or explore food data from the Department of Agriculture (2)? 
Please input a 1 or 2 below:


 2


Would you like data as a CSV (1), JSON (2), or SQL database (3)?
Please input a 1,2, or 3 below


 1


Please type the food you want to learn about


 yam


You have requested to see USDA information on yam as a CSV file.
Name: YAM
Ingredients: YAM AND SODIUM BISULFITE USED AS A PRESERVATIVE.
Country Market: United States
Serving Size: 150.0g
Would you like to delete the ingredients list? Answer 1 for yes, or type in any response for no


 1


Your dataset has one row and 4 columns
Name: YAM
Country Market: United States
Serving Size: 150.0g
Processing is complete. There are 1 row(s) in this dataset and 4 column(s).
File name: YAM.csv


,Unnamed: 0,Name,Country Market,Serving Size,Serving Size Unit
0,0,YAM,United States,150.0,g


API request successful!
